# PREPROCESARE VENUE
### 1. Extrage venues in litere mici
### 2. Elimina punctuatia 
### 3. Elimina cuvintele care nu sunt in engleza
### 4. Elimina stopwords
### 5. Calculez numarul de aparitii pentru fiecare cuvant
### 6. Elimin cuvinte cu aparitie mai mare de 15 si de 10
### 7. Elimin cuvinte legate de locatie

In [1]:
import json # pentru fisier
import string # pentru lower si punctuation
from nltk.corpus import wordnet # pentru verificare limbii
from nltk.corpus import stopwords # pentru eliminare stopwords
import itertools # pentru sortare
from collections import Counter 
from nltk.corpus import wordnet # To get words in dictionary with their parts of speech
from nltk.stem import WordNetLemmatizer # lemmatizes word based on it's parts of speech

In [22]:
def extrage_venues(file):
    venues = []
    readFile = open(file,'r',encoding='utf-8')

    for line in readFile:
        current_paper = json.loads(line)
        current_venue = current_paper['venue']
        venues.append(current_venue.lower())
    readFile.close()
    return venues

def elimina_punctuatia(venues):
    new_venues = []
    
    for venue in venues:
        
        word = ""
        new_current_venue = []
        
        for ch in venue:
            if ch == ' ' or ch =='-':
                new_current_venue.append(word)
                word = ""
            else:
                if ch not in string.punctuation:
                    word += ch
        new_current_venue.append(word)
        new_venues.append(new_current_venue)
        
    return new_venues

def elimina_non_eng(venues):
    new_venues = []
    
    for venue in venues:
        new_current_venue = []
        for elem in venue:
            if wordnet.synsets(elem):
                new_current_venue.append(elem)
        new_venues.append(new_current_venue)
    return new_venues

def elimina_stopwords(venues):
    new_venues = []
    stop_words = set(stopwords.words('english')) 
    for venue in venues:
        new_current_venue = []
        for elem in venue:
            if elem not in stop_words:
                new_current_venue.append(elem)
        new_venues.append(new_current_venue)
    return new_venues

def elimina_country(venues, country_list):
    new_venues = []
    
    for venue in venues:
        new_current_venue = []
        for word in venue:
            if word not in country_list:
                new_current_venue.append(word)
        new_venues.append(new_current_venue)
    return new_venues
            

def calculeaza_nr_aparitii(venues):
    # Calculez numarul total de cuvinte
    words_dict = {}
    word_list = []
    for venue in venues:
        for elem in venue:
            try:
                words_dict[elem] += 1
            except:
                words_dict[elem] = 1
                word_list.append(elem)
                
    return words_dict, word_list

def calculeaza_nr_aparitii_set_date(venues):
    dict_venues = {}
    for list_ in venues:
        for elem in list_:
            dict_venues[elem] = 0
    for i in range(0,4):
        r = open('../../dblp-ref/dblp-final-'+str(i)+'.json')
        for line in r:
            art_crt = json.loads(line)
            venue  = art_crt['venue']
            venue = venue.split(' ')
            for word in venue:
                print (word)
                break
        r.close()
        break

def elimina_cuv_dupa_aparitie(word_dict, word_list,frecv_max, frev_min):
    new_word_list = []
    
    for word in word_list:
        if word_dict[word] <= frecv_max and word_dict[word] >= frev_min:
            new_word_list.append(word)
    return new_word_list

def elimina_venue(venues,word_list_reduced):
    new_venues = []
    for venue in venues:
        new_current_venue = []
        for elem in venue:
            if elem in word_list_reduced:
                new_current_venue.append(elem)
        new_venues.append(new_current_venue)
    return new_venues


def get_pos( word ):
    w_synsets = wordnet.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0]

def lemmatize_word(word):
    pos = get_pos(word)
    wnl = WordNetLemmatizer()
    return (wnl.lemmatize( word, pos ))
    
    
def lemmatizare_cuvinte(venues):
    new_venues = []
    for venue in venues:
        new_current_venue = []
        for elem in venue:
            new_current_venue.append(lemmatize_word(elem))
        new_venues.append(new_current_venue)
    return new_venues

## Functii auxiliare

In [5]:
# Functii auxiliare
def print_ll(listaDeListe, numeFisier):
    numeFisier += ".json"
    wFile = open(numeFisier,'w')
    wFile.write(json.dumps(listaDeListe))
    wFile.close()

def pretty_print_ll(listaDeListe, numeFisier):
    numeFisier += "_pp.txt"
    wFile = open(numeFisier,'w')
    for lista in listaDeListe:
        for elem in lista:
            wFile.write(elem)
            wFile.write(' ')
        wFile.write('\n')
    wFile.close()

def pretty_print_l(lista, numeFisier):
    numeFisier += "_ppl.txt"
    wFile = open(numeFisier,'w')
    for elem in lista:
        wFile.write(elem)
        wFile.write('\n')
    wFile.close()
    
def print_dict(dict_, numeFisier):
    numeFisier += ".json"
    wf = open(numeFisier,"w")
    wf.write(json.dumps(dict_))
    wf.close()
    
def eliminare_duplicate_si_vide(lista):
    lista.sort()
    sort_lista = list(lista for lista,_ in itertools.groupby(lista))
    if sort_lista[0] == []:
        return sort_lista[1:]
    return sort_lista
    
    
def maxim_aparitii(word_list, word_dict):
    max_ap = 0
    for elem in word_list:
        if word_dict[elem] > max_ap:
            max_ap = word_dict[elem]
    return max_ap

def sortare_dictionar(word_dict):
    word_dict_sorted = {}
    for w in sorted(word_dict, key=word_dict.get, reverse=True):
        word_dict_sorted[w] = word_dict[w]
    return word_dict_sorted

def append_listeDeListe(new_list, old_list):
    for elem in new_list:
        old_list.append(elem)
    return old_list

In [9]:
venues0 = extrage_venues('../../dblp-ref/dblp-final-0.json')

In [5]:
venues0 = extrage_venues('../../dblp-ref/dblp-final-0.json')
venues1 = extrage_venues('../../dblp-ref/dblp-final-1.json')
venues2 = extrage_venues('../../dblp-ref/dblp-final-2.json')
venues3 = extrage_venues('../../dblp-ref/dblp-final-3.json')

venues = append_listeDeListe(venues0,venues1)
venues = eliminare_duplicate_si_vide(venues)


venues = append_listeDeListe(venues2,venues)
venues = eliminare_duplicate_si_vide(venues)

venues = append_listeDeListe(venues3,venues)
venues = eliminare_duplicate_si_vide(venues)

print(len(venues))

4264


In [6]:
with open('country_list.json') as json_file:  
    country_list = json.load(json_file)

## Preprocesare venues pasi

In [7]:
venues = elimina_punctuatia(venues)
venues = elimina_non_eng(venues)
venues = elimina_stopwords(venues)
venues = elimina_country(venues,country_list)
venues = eliminare_duplicate_si_vide(venues)
print(len(venues))

3785


In [25]:
print_ll(venues,"all_venues1_lemma")

In [2]:
r = open('all_venues1_lemma.json','r')
venues = json.load(r)
r.close()

In [6]:
word_dict, word_list = calculeaza_nr_aparitii(venues)

In [23]:
print(venues[0])
calculeaza_nr_aparitii_set_date(venues)

['3d', 'research']
international
visual
pattern
joint
operating
international
The
international
conference
Applied
intelligent
international
formal
the
international
arXiv:
international
symposium
doctoral
conference
international
international
pacific
simulation,
international
string
The
north
conference
text
intelligent
international
international
international
national
Theoretical
Synthese
wireless
national
artificial
north
theory
rough
european
conference
advanced
arXiv:
similarity
international
principles
active
national
empirical
information
Theoretical
international
international
international
hybrid
Applied
genetic
Journal
international
usenix
arXiv:
international
Lecture
affective
international
summer
international
international
european
Journal
national
international
international
parallel
agent
pacific
knowledge
national
international
parallel
soft
discovery
international
asian
software
Reliable
international
international
international
information
international
internationa

conference
Fuzzy
medical
international
international
medical
european
international
conference
artificial
mexican
statistical
cross
national
international
International
Revista
international
Journal
national
national
arXiv:
knowledge
advances
computer
international
international
knowledge
mexican
Transactions
learning
International
Journal
advances
arXiv:
national
medical
Computing
medical
formal
international
european
conference
Psychnology
international
international
australasian
the
very
conference
intelligent
brazilian
Annales
national
computer
Journal
international
text
web
International
hybrid
national
international
workshop
computer
database
symposium
Computers
conference
database
international
international
international
Lecture
conference
international
computer
Communications
european
scandinavian
intelligent
Operational
automation,
european
international
simulation
international
international
international
Journal
international
international
asian
international
international


SIAM
international
parallel
adaptive
Journal
advances
logic
international
principles
Computing
asian
international
usenix
knowledge
industrial
knowledge
computing
national
Annals
principles
international
conference
pattern
Informatica
genetic
computational
european
international
Journal
wireless
international
international
smart
international
applications
International
distributed
international
iberian
web
rough
Lecture
conference
international
international
Management
national
conference
international
mathematical
medical
wired
international
international
canadian
parallel
national
international
arXiv:
Journal
arXiv:
ubiquitous
Communications
conference
Queueing
arXiv:
Lecture
usenix
computational
international
conference
international
soft
theory
international
requirements
Communications
international
knowledge
international
adaptive
advances
mathematical
asian
Journal
usenix
human
national
data
international
uncertainty
asia-pacific
static
international
international
formal
asian
in

arXiv:
mathematical
pacific
international
international
Human-centric
arXiv:
international
scandinavian
medical
computer
british
information
international
mathematical
international
international
high
international
computer
international
international
Journal
network
international
international
artificial
human
international
national
operating
Theoretical
Lecture
national
international
national
Journal
algorithm
international
formal
international
international
international
empirical
Computing
international
international
Lecture
International
Journal
similarity
computer
international
web
Lecture
Engineering
pacific
Lecture
pacific
european
IEEE
north
european
european
artificial
national
artificial
iberoamerican
arXiv:
national
Computing
Security
data
Quantum
international
european
Communications
international
artificial
scalable
international
international
national
human
automation,
Computational
international
automated
portuguese
text
arXiv:
communications
arXiv:
pacific
internationa

parallel
australasian
international
british
artificial
arXiv:
international
workshop
Distributed
european
national
human
international
automation,
passive
cellular
international
international
intelligent
medical
Computing
pacific
international
national
empirical
vision
computer
international
international
international
international
international
international
european
european
international
european
international
automated
national
international
Lecture
joint
international
international
hybrid
advances
advanced
pacific
Computing
european
asian
conference
international
usenix
international
pacific
Abdominal
international
asian
intelligent
australasian
data
international
computational
european
international
international
the
international
international
parallel
rough
autonomous
medical
database
international
Management
arXiv:
computing
arXiv:
Journal
Information
european
Journal
medical
international
artificial
international
hybrid
international
Journal
international
european
medical
in

...

web
conference
international
national
international
european
web
International
fundamental
international
mexican
advances
international
financial
medical
arXiv:
International
modeling
Computers
international
Mathematics
european
international
international
programming
international
national
international
Lecture
international
personal,
arXiv:
Transactions
computational
international
international
european
automated
computing
applications
information
Applied
principles
ieee
mathematical
tools
australian
arXiv:
robot
foundations
embedded
symposium
graphics
european
Journal
power
Lecture
Lecture
Lecture
Security
multimedia
asian
information
international
ieee
computer,
next
arXiv:
BMC
mathematical
conference
arXiv:
compiler
web
international
international
cellular
international
formal
algebraic
international
high
Mis
Analog
Higher-order
Sensors
industrial
international
Lecture
discovery
computing
statistical
web
database
Lecture
Fundamenta
european
international
international
Journal
high

arXiv:
language
graphics
adaptive
applied
integrated
congress
international
international
computer
Discrete
international
algebraic
international
Lecture
Lecture
Wireless
Lecture
IEEE
biologically
modeling,
combinatorial
web
international
Lecture
Journal
international
Computational
medical
symposium
public
conference
australian
european
applications
international
Fundamenta
kommunikation
Lecture
IEEE
flexible
trust
ifip
personal,
algorithmic
international
international
ieee
next
fuzzy
symposium
new
International
web-age
international
international
international
international
international
software
australian
Australasian
european
Lecture
algorithmic
adaptive
asia
european
soft
International
international
Lecture
document
Information
signal
international
international
Journal
canadian
Lecture
international
international
international
international
pacific-rim
arXiv:
international
international
international
Lecture
international
foundations
Lecture
usenix
computer
intelligent
ieee
inter

Designs,
international
international
arXiv:
international
european
picture
decision
Lecture
asian
foundations
Statistics
computational
Mathematical
Lecture
partial
conference
Remote
international
Lecture
Lecture
virtual
international
systems,
cryptology
international
field-programmable
Lecture
international
international
database
Lecture
international
artificial
The
Machine
international
intelligent
cross-language
Journal
conference
eurographics
Mathematical
Lecture
arXiv:
arXiv:
pacific
logical
pacific
Electronic
ubiquitous
international
international
european
high
international
international
working
discrete
wireless
Australasian
text,
design
conference
fundamental
pacific
pattern
systems,
pacific
Lecture
australian
next
advances
international
Lecture
verified
Computing
automation,
Mathematics
vlsi
Lecture
Security
international
international
international
arXiv:
mexican
international
international
european
international
international
international
fuzzy
international
web
advances
fo

databases
european
Management
Lecture
cross
arXiv:
international
international
web
international
vehicular
international
european
applied
grid
summer
international
international
business
conference
international
database
international
mexican
international
arXiv:
international
International
international
field-programmable
Journal
international
advanced
conference
advances
international
computing
international
european
international
BMC
international
international
Lecture
Transactions
Genetic
international
Electronic
international
international
Lecture
graphics
Australasian
distributed
location
international
international
international
intelligent
international
distributed
conference
parallel
Lecture
Lecture
arXiv:
Advances
embedded
distributed
Lecture
international
Lecture
robotics,
computational
Lecture
mAbs
ieee
international
Journal
workshop
international
Security
international
wireless
Lecture
international
asian
arXiv:
agent
Lecture
Lecture
Electronic
mathematical
european
comput

KeyboardInterrupt: 

In [8]:
for elem in word_dict:
    print(word_dict[elem])
    break

2


In [7]:
print(len(word_list))
print(len(word_dict))

1360
1360


In [11]:
word_dict_sorted = sortare_dictionar(word_dict)
print(len(word_dict_sorted))

1484


In [12]:
word_list_reduced15 = elimina_cuv_dupa_aparitie(word_dict, word_list,15,1)
word_list_reduced10 = elimina_cuv_dupa_aparitie(word_dict, word_list,10,1)
word_list_reduced5 = elimina_cuv_dupa_aparitie(word_dict, word_list,5,1)
print(len(word_list_reduced15))
print(len(word_list_reduced10))
print(len(word_list_reduced5))

1333
1267
1143


## Elimina cuvintele

In [30]:
venues_reduced_15 = elimina_venue(venues,word_list_reduced15)
venues_reduced_10 = elimina_venue(venues,word_list_reduced10)
venues_reduced_5 = elimina_venue(venues,word_list_reduced5)

In [31]:
print(len(venues_reduced_15))
print(len(venues_reduced_10))
print(len(venues_reduced_5))

3797
3797
3797


In [32]:
venues_final_15 = eliminare_duplicate_si_vide(venues_reduced_15)
venues_final_10 = eliminare_duplicate_si_vide(venues_reduced_10)
venues_final_5 = eliminare_duplicate_si_vide(venues_reduced_5)
print(len(venues_final_15))
print(len(venues_final_10))
print(len(venues_final_5))

1501
1289
1015


In [33]:
print_ll(venues_final_15, "venues_reduced_15_lemma")
print_ll(venues_final_10, "venues_reduced_10_lemma")
print_ll(venues_final_5, "venues_reduced_5_lemma")

In [48]:
pretty_print_l(word_list_reduced10, "word_list_reduced10")

In [34]:
print(word_list_reduced10[0:10])

['3d', 'abdominal', 'ac', 'academic', 'radiology', 'academy', 'accelerator', 'access', 'accessibility', 'acid']
